# Covid-19 unemployment benefits, payroll tax cuts, and UBI

## Setup

In [1]:
import numpy as np
import pandas as pd
import microdf as mdf

Note that `FFPOS` is in family dataset, `PH_SEQ`.

Per https://www2.census.gov/programs-surveys/demo/datasets/income-poverty/time-series/data-extracts/final-cps-pums-v3-ll-layout.pdf

In [2]:
p = pd.read_csv('/home/mghenis/MaxGhenis/datarepo/pppub19.csv.gz')
f = pd.read_csv('/home/mghenis/MaxGhenis/datarepo/ffpub19.csv')
h = pd.read_csv('/home/mghenis/MaxGhenis/datarepo/hhpub19.csv')

In [3]:
cps = pd.read_csv('/home/mghenis/MaxGhenis/datarepo/cpsmar2018.csv.gz')

Merge

In [6]:
p.columns = p.columns.str.lower()
f.columns = f.columns.str.lower()
h.columns = h.columns.str.lower()

In [7]:
h['ph_seq'] = h.h_seq
p['fh_seq'] = p.ph_seq
p['ffpos'] = p.phf_seq

In [8]:
h.h_hhtype

0        3
1        3
2        3
3        1
4        3
        ..
94628    1
94629    1
94630    1
94631    1
94632    1
Name: h_hhtype, Length: 94633, dtype: int64

In [10]:
merged = p.merge(h[h.h_hhtype == 1], on='ph_seq')
merged = merged.merge(f, on=['fh_seq', 'ffpos'], how='left')

In [11]:
merged.shape

(180101, 1021)

In [14]:
merged[cps.columns]

KeyError: "['iahityp', 'tretval1', 'paid', 'hmcare', 'i_otyp', 'ahiper', 'otyp_5', 'tretval2', 'pelndad', 'paidcyna', 'i_retsc1', 'tcare_val', 'i_ostper', 'i_oth', 'axspouse', 'hilin1', 'tot_inc', 'earned_inc', 'hilin2', 'hi', 'prityp', 'cov_gh', 'i_out', 'hchamp', 'i_retvl2', 'mon', 'hi_yn', 'i_dephi', 'ahityp5', 'h_type', 'i_pout', 'i_ostyp', 'tint_val', 'fwifeidx', 'ch_hi', 'cov_hi', 'ret_yn', 'i_paid', 'i_edtyp2', 'pxdadtyp', 'i_hiout', 'p_flag', 'pedadtyp', 'hiout', 'othstyp1', 'othstyp6', 'sskidyna', 'ahityp3', 'unearned_inc', 'prop_tax', 'i_hi', 'pilin1', 'pxlndad', 'pout', 'i_edtyp1', 'hret_yn', 's_flag', 'care', 'statetax_bc', 'ahityp1', 'oth', 'axrrp', 'pelnmom', 'othstyp5', 'tanf_val', 'otyp_2', 'h_idnum1', 'out', 'ret_sc1', 'i_retvl1', 'i_mon', 'i_retsc2', 'ahityp6', 'champ', 'othstyp2', 'alimony', 'ssikidyn', 'hretval', 'hityp', 'finc_ret', 'hhpos', 'a_parent', 'iahiper', 'hhid', 'othstyp4', 'hrpaidcc', 'ahityp2', 'pensions_annuities', 'othstyp3', 'rtm_val', 'depriv', 'Unn

In [12]:
cps.shape

(180084, 709)

In [34]:
p_raw = pd.read_csv('/home/mghenis/MaxGhenis/datarepo/pppub19.csv.gz',
                    usecols=['PH_SEQ', 'PPPOS',
                             'SPM_ID', 'SPM_POVTHRESHOLD', 'SPM_RESOURCES'])

In [37]:
cps = pd.read_csv('/home/mghenis/MaxGhenis/datarepo/cpsmar2018.csv.gz',
                  usecols=['lkweeks', 'marsupwt',
                           'ph_seq', 'pppos', 'h_seq', 'a_lineno', 'ffpos'])

In [38]:
tu = pd.read_csv('/home/mghenis/MaxGhenis/datarepo/cps_taxcalc_2018.csv.gz')

## Preprocess

In [41]:
spm.columns = spm.columns.str.lower()

In [50]:
PERSON_IDS = ['ph_seq', 'pppos']
person = spm.merge(cps, on=PERSON_IDS)

FPUC_MAX_WEEKS = 17  # April to July.
FPUC2_MAX_WEEKS = 17 + 22  # Adding August to December.
FPUC_WEEKLY_BEN = 600
person['fpuc'] = FPUC_WEEKLY_BEN * np.minimum(person.lkweeks, FPUC_MAX_WEEKS)
person['fpuc2'] = FPUC_WEEKLY_BEN * np.minimum(person.lkweeks,
                                               FPUC2_MAX_WEEKS)

In [59]:
TU_IDS = ['h_seq', 'a_lineno', 'ffpos']
tu_ui = person.groupby(TU_IDS)[['fpuc', 'fpuc2']].sum().reset_index()

Verify these are the correct identifiers for tax units.

In [58]:
assert tu.duplicated(TU_IDS).sum() == 0

In [61]:
tu2 = tu.merge(tu_ui, on=TU_IDS)

In [70]:
cps[PERSON_IDS]

,ph_seq,pppos
0,2,41
1,4,41
2,4,42
3,4,43
4,4,44
...,...,...
180079,92138,41
180080,92138,42
180081,92139,41
180082,92139,42


In [69]:
spm[PERSON_IDS]

,ph_seq,pppos
0,4,41
1,6,41
2,7,41
3,8,41
4,8,42
...,...,...
180096,94632,44
180097,94632,45
180098,94632,46
180099,94633,41


In [68]:
person.shape[0]

90807

In [62]:
assert tu.shape[0] == tu2.shape[0]

AssertionError: 

In [48]:
(person.fpuc2 * person.marsupwt).sum() / 1e9

65.46867936

In [9]:
tu.columns

Index(['e00200', 'e00200p', 'e00200s', 'e00900', 'e00900p', 'e00900s',
       'e02100', 'e02100p', 'e02100s', 'e00600', 'e01500', 'e00800', 'e02400',
       'e02300', 'ssi_ben', 'tanf_ben', 'vet_ben', 'snap_ben', 'housing_ben',
       'age_head', 'age_spouse', 'blind_head', 'fips', 'h_seq', 'a_lineno',
       'ffpos', 's006', 'FLPDYR', 'EIC', 'DSI', 'MARS', 'XTOT', 'nu18',
       'n1820', 'n21', 'nu06', 'nu13', 'n24', 'elderly_dependents', 'f2441',
       'e18400', 'blind_spouse', 'e00650', 'e00300', 'e00400', 'e01700',
       'e01100', 'e01400', 'e03300', 'e03270', 'e32800', 'e17500', 'e20400',
       'e03240', 'e19200', 'e18500', 'mcaid_ben', 'mcare_ben', 'other_ben',
       'e19800', 'e20100', 'e03210', 'e03150', 'RECID', 'agi_bin', 'pencon_p',
       'pencon_s'],
      dtype='object')